In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/ML/data/Dataset/")

In [ ]:
# Function to load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize images to 224x224 for VGG16
            images.append(img)  # Keep original image for visualization
            labels.append(label)
    return images, labels

In [ ]:
cat_images, cat_labels = load_images_from_folder(r'/content/drive/My Drive/ML/data/Dataset/test1', 0)
dog_images, dog_labels = load_images_from_folder(r'/content/drive/My Drive/ML/data/Dataset/train', 1)

In [ ]:
images = cat_images + dog_images
labels = cat_labels + dog_labels

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
def extract_features(image):
    img = img_to_array(image)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = base_model.predict(img)
    return features.flatten()

In [ ]:
X = np.array([extract_features(img) for img in images])
y = np.array(labels)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 71ms/step


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score



# Pipeline for scaling and SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# Parameter grid for Grid Search
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [1, 0.1, 0.01, 0.001],
    'svc__kernel': ['rbf', 'poly', 'sigmoid']
}

# Grid Search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and accuracy
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test set prediction
y_pred = grid_search.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))


NameError: name 'X_train' is not defined

In [ ]:
# Use the fitted grid_search object to predict
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
def plot_predictions(original_images, predictions, true_labels):
    fig, axes = plt.subplots(2, 5, figsize=(12, 6))
    axes = axes.ravel()
    for i in range(10):
        axes[i].imshow(cv2.cvtColor(original_images[i], cv2.COLOR_BGR2RGB))
        axes[i].set_title(f'Pred: {predictions[i]}\nTrue: {true_labels[i]}')
        axes[i].axis('off')
    plt.subplots_adjust(hspace=0.5)
    plt.show()

In [ ]:
plot_predictions(images[:100], y_pred[:100], y_test[:100])